In [ ]:
import zipfile
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from PIL import Image

In [ ]:
# Step 1: Unzip the file
zip_file_path = '/content/AI_lab_generated.zip'
extract_to = '/content/AI_images'  # Directory to extract the contents

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

In [ ]:
# Step 2: Define the paths and load the dataset
data_root = os.path.join(extract_to, "AI_lab_generated")

In [ ]:
# Step 3: Data Preparation
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Use ImageFolder to load data
dataset = datasets.ImageFolder(root=data_root, transform=transform)

# Split into train/test sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print("Classes:", dataset.classes)  # Should output: ['artistic_rainy', 'artistic_sunny', 'realistic_rainy', 'realistic_sunny']

Classes: ['artistic_rainy', 'artistic_sunny', 'realistic_rainy', 'realistic_sunny']


In [ ]:
# Step 4: Define CNN Model for Weather Classification (Rainy/Sunny)
class WeatherClassifier(nn.Module):
    def __init__(self):
        super(WeatherClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, 1)
        self.conv2 = nn.Conv2d(16, 32, 3, 1)
        self.fc1 = nn.Linear(32 * 30 * 30, 128)
        self.fc2 = nn.Linear(128, 2)  # 2 classes: rainy and sunny

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.max_pool2d(x, 2)
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
# Step 5: Define CNN Model for Style Classification (Artistic/Realistic)
class StyleClassifier(nn.Module):
    def __init__(self):
        super(StyleClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, 1)
        self.conv2 = nn.Conv2d(16, 32, 3, 1)
        self.fc1 = nn.Linear(32 * 30 * 30, 128)
        self.fc2 = nn.Linear(128, 2)  # 2 classes: artistic and realistic

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.max_pool2d(x, 2)
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
# Step 6: Define the Models, Optimizers, and Loss Function
weather_model = WeatherClassifier()
style_model = StyleClassifier()

optimizer = optim.Adam(weather_model.parameters(), lr=0.001)
style_optimizer = optim.Adam(style_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [ ]:
# Step 7: Define the model training function
def train_model(model, train_loader, optimizer, criterion, num_epochs=10, classifier_type='weather'):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            if classifier_type == 'weather':
                # Convert labels to binary: 0 for rainy, 1 for sunny
                labels = [0 if 'rainy' in dataset.classes[label] else 1 for label in labels]
            elif classifier_type == 'style':
                # Convert labels to binary: 0 for artistic, 1 for realistic
                labels = [0 if 'artistic' in dataset.classes[label] else 1 for label in labels]

            labels = torch.tensor(labels)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}")

In [ ]:
# Step 8: # Train weather classifier
print("\nTraining Weather Classifier (Rainy/Sunny)...")
train_model(weather_model, train_loader, optimizer, criterion, num_epochs=10, classifier_type='weather')




Training Weather Classifier (Rainy/Sunny)...
Epoch 1, Loss: 0.5861966162919998
Epoch 2, Loss: 0.171401164184014
Epoch 3, Loss: 0.04625445449103912
Epoch 4, Loss: 0.01564870000584051
Epoch 5, Loss: 0.004719881758016224
Epoch 6, Loss: 0.0013302478667659063
Epoch 7, Loss: 0.0005243842521546563
Epoch 8, Loss: 0.0003516367998296725
Epoch 9, Loss: 0.0001405849416187266
Epoch 10, Loss: 0.00010389551501551371


In [ ]:
# Step 9: Train style classifier
print("\nTraining Style Classifier (Artistic/Realistic)...")
train_model(style_model, train_loader, style_optimizer, criterion, num_epochs=10, classifier_type='style')


Training Style Classifier (Artistic/Realistic)...
Epoch 1, Loss: 0.535306935509046
Epoch 2, Loss: 0.1259213094599545
Epoch 3, Loss: 0.022945523727685213
Epoch 4, Loss: 0.015788223575024556
Epoch 5, Loss: 0.006957649165997282
Epoch 6, Loss: 0.00040622339050363127
Epoch 7, Loss: 0.0011646686277041833
Epoch 8, Loss: 0.0002313562032062085
Epoch 9, Loss: 1.7104969818622823e-05
Epoch 10, Loss: 1.3409113800359288e-05


In [ ]:
# Step 10: Save the Weather Trained Model
torch.save(weather_model.state_dict(), 'weather_classifier_model.pth')

In [ ]:
# Step 11: Save the Weather Trained Model
torch.save(style_model.state_dict(), 'style_classifier_model.pth')

In [ ]:
#Test nodel for weather prediction
# Step1: put the model in evaluation mode
weather_model.eval()

# Step2: find the image for prediction
image_path = '/content/model_test/test_weather.png'
image = Image.open(image_path)
image = transform(image)
image = image.unsqueeze(0)

# Step 3: Make predictions
with torch.no_grad():
    output = weather_model(image)
    probabilities = torch.softmax(output, dim=1)
    predicted_class = torch.argmax(probabilities, dim=1).item()

# Step 4: Output the prediction
if predicted_class == 0:
    print("The predicted weather is: Rainy")
else:
    print("The predicted weather is: Sunny")

The predicted weather is: Sunny


In [ ]:
#Test the style model (predict image style)

# Step1: put model in eval mode
style_model.eval()
# Step2: find the image for testing style
image_path = '/content/model_test/test_style.png'
image = Image.open(image_path)
image = transform(image)
image = image.unsqueeze(0)

# Step 3: Make predictions
with torch.no_grad():
    output = weather_model(image)
    probabilities = torch.softmax(output, dim=1)
    predicted_style = torch.argmax(probabilities, dim=1).item()

# Step 4: Output the prediction
if predicted_style == 0:
    print("The predicted weather is: artistic")
else:
    print("The predicted weather is: realistic")

The predicted weather is: artistic
